# Affine vs Projective coordinates benchmark
I stumbled upon an article online about eliminating division in your computations (which we can do in $\mathbb{P}^2$ by allowing z to balloon). After re-deriving the formulas for it I wanted to see what the actual speed comparison was.

So far (looks like there are still bugs to squash) it looks like it's much faster to just divide and do the usual computations in $\mathbb{A}^2$.

---
First I did some tests to see how much slower division was than multiplication:

In [27]:
%timeit 3429785342985/2343243245532

11.3 ns ± 0.164 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [28]:
%timeit 3429785342985*2343243245532

11.3 ns ± 0.233 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [37]:
import random
import time

ITERS = 100000
ROUNDS = 26

print("exponent\t mult/div ratio")

total = 0
for i in range(ROUNDS):
    e = i+5
    x = random.randint(10**i,10**(i+1))
    y = random.randint(10**i,10**(i+1))
    
    start = time.time()
    for _ in range(ITERS):
        z = x * y
    m = time.time()-start
    
    start = time.time()
    for _ in range(ITERS):
        z = x/y
    d = time.time()-start
    
    total += m/d
    print("%s\t\t%s" % (e,m/d))
    
print("Average: \t%s" % (total/ROUNDS))

exponent	 mult/div ratio
5		2.0073077219977873
6		1.7327504608180142
7		1.1855462096795326
8		1.1088497231954213
9		1.0639706989112439
10		1.0605976687394578
11		1.0211238267944225
12		1.0939253793390147
13		1.105144088338271
14		1.1858163826278174
15		1.1944904772197804
16		1.2272738365424507
17		1.122423408160384
18		1.2324040384569046
19		1.2565591185335243
20		1.0980558874896589
21		0.43517475633441827
22		0.45015853233876435
23		0.4637305248498564
24		0.3905761199838345
25		0.5279619486883829
26		0.4987221315932572
27		0.459920790355573
28		0.4845837887388499
29		0.4556765028834266
30		0.39893948191312983
Average: 	0.9331416732508914


In [3]:
def f(x,y,z):
    s = 3*x**2 + 14*z**2
    newZ = 8*(y**3)*(z**4)
    t = x*z*(y**2)
    newX = 2*y*z*( s**2 - 8*t )
    
    newY = s**3 - 12*s*t + y*t
    return (newX/newZ, newY/newZ, 1)

In [9]:
def g(x,y,z):
    x = x/z
    y = y/z
    s = (3*x**2+14)/(2*y)
    newX = s**2-2*x
    newY = s*(newX - x)+y
    newZ = 1
    return (newX, newY, newZ)

In [5]:
(x,y,z) = (15,232311,2342)

In [6]:
%timeit f(x,y,z)

2.93 µs ± 76.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [10]:
%timeit g(x,y,z)

549 ns ± 3.69 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [11]:
print(f(x,y,z))
print(g(x,y,z))

(-3.3430682314207614e-06, -4.2876447291432337e-07, 1)
(-0.007829465797987425, 99.1924199153304, 1)


Uh oh. Made a mistake in my computations.

In [30]:
import math
math.log(10**21)/math.log(2)

69.7604899926346